In [2]:
import json
from datetime import datetime

# File paths
logical_path = "/Users/duchuyta/Downloads/SAP-CONUHACKS-IX/Data/Output/model1_logical.json"
supposed_true_path = "/Users/duchuyta/Downloads/SAP-CONUHACKS-IX/Data/Output/model1_supposedTrue.json"

# Load model1_logical.json
with open(logical_path, 'r') as f:
    logical_data = json.load(f)

# (Optional) Load the supposedTrue template if needed for reference
with open(supposed_true_path, 'r') as f:
    supposed_true_data = json.load(f)

# Rebuild the JSON structure to match the supposedTrue template
# For summary: use available fields or compute from by_severity if possible
by_severity = logical_data.get("by_severity", {})
handled = by_severity.get("handled", {})
missed = by_severity.get("missed", {})

# If original summary doesn't have all needed keys, fallback to computed values
total_fires_handled = logical_data.get("summary", {}).get("total_fires_handled", 
                              sum(handled.get(level, 0) for level in ['low','medium','high']))
total_fires_missed = logical_data.get("summary", {}).get("total_fires_missed", 
                              sum(missed.get(level, 0) for level in ['low','medium','high']))

# For units_used and total_units, try to use existing values; otherwise, default to total_fires_handled
units_used = logical_data.get("summary", {}).get("units_used", total_fires_handled)
total_units = logical_data.get("summary", {}).get("total_units", total_fires_handled)

new_summary = {
    "total_fires_handled": total_fires_handled,
    "total_fires_missed": total_fires_missed,
    "units_used": units_used,
    "total_units": total_units
}

# For costs, simply use the available operational and damage costs
costs = logical_data.get("costs", {})
# Ensure that 'operational' and 'damage' are present, and compute total if necessary.
operational = costs.get("operational", 0)
damage = costs.get("damage", 0)
new_costs = {
    "operational": operational,
    "damage": damage,
    "total": operational + damage
}

# For assignments, if a "schedule" exists in the logical data, rebuild assignments using longitude and latitude.
# Otherwise, keep the existing assignments.
new_assignments = []
if "schedule" in logical_data and isinstance(logical_data["schedule"], list):
    for event in logical_data["schedule"]:
        # Extract fire start date (YYYY-MM-DD)
        fire_date = event.get("fire_start_time", "")[:10] if event.get("fire_start_time") else "unknown_date"
        severity = event.get("severity", "unknown")
        # Use longitude and latitude instead of a generic location
        latitude = event.get("latitude", "unknown")
        longitude = event.get("longitude", "unknown")
        # Prepare assigned units string
        assigned_units = event.get("assigned_units", [])
        unit_str = ", ".join(map(str, assigned_units)) if assigned_units else "unassigned"
        event_id = event.get("event_id", "")
        assignment_str = f"{fire_date}: {severity} fire at ({latitude}, {longitude}) -> {unit_str} (ID: {event_id})"
        new_assignments.append(assignment_str)
else:
    # Fallback: use the existing assignments as-is
    new_assignments = logical_data.get("assignments", [])

# Set the current timestamp
timestamp = datetime.now().isoformat()

# Combine everything into the new structure
new_data = {
    "summary": new_summary,
    "by_severity": {
        "handled": {
            "low": handled.get("low", 0),
            "medium": handled.get("medium", 0),
            "high": handled.get("high", 0)
        },
        "missed": {
            "low": missed.get("low", 0),
            "medium": missed.get("medium", 0),
            "high": missed.get("high", 0)
        }
    },
    "costs": new_costs,
    "assignments": new_assignments,
    "timestamp": timestamp
}

# Write the updated JSON back to model1_logical.json
with open(logical_path, 'w') as f:
    json.dump(new_data, f, indent=2)

print("model1_logical.json has been updated to match the model1_supposedTrue.json template (using longitude and latitude).")

model1_logical.json has been updated to match the model1_supposedTrue.json template (using longitude and latitude).
